In [29]:
import os
import sys
import pickle

import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table
import astropy.units as u
from astropy.cosmology import WMAP9
import astropy.constants as c

from matplotlib import pyplot as plt
from cycler import cycler
%matplotlib

import spectroscopy

Using matplotlib backend: Qt5Agg


In [30]:
sys.path.append('../code')
import scale_merge_spec

In [31]:
FIG_DIR = '../figures'

In [32]:
CAT_DIR = '../astropy_catalogs/'

# Plot Scaled Spectra to Determine which to combine

In [33]:
def plot_scaled_spectra(sn_dir, sn, tbdata):
    plt.ioff()
    cycler_net = cycler(color='bgrcmyk')+cycler(linestyle=':::::::')
    for date in np.unique(tbdata['Obs. Date']):
        print("'{}':{}'filename':{}".format(date, '{','('))
        indx = (tbdata['Obs. Date']==date) & (tbdata['Quality']=='Final')
        if len(tbdata[indx]) > 0:
            spec_list = []
            for irow in tbdata[indx]:
                if not hasattr(irow['Fits File'], '_mask'):
                    if 'WiFeS' in irow['Instrument']:
                        spectrum = scale_merge_spec.fits_spec(sn_dir, irow['Fits File'], ext=1)
                    else:
                        spectrum = scale_merge_spec.fits_spec(sn_dir, irow['Fits File'])
                elif not hasattr(irow['Ascii File'], '_mask'):
                    spectrum = scale_merge_spec.ascii_spec(sn_dir, irow['Ascii File'])
                else:
                    print('Cannot read id={}, date={}'.format(irow['Id'], date))
                    continue
                spec_list.append(spectrum)
                print("'{}',".format(spectrum.filename))
            template_spec = spec_list[0]
            if len(spec_list)>1:
                for spec in spec_list[1:]:
                    template_spec, scaled_spec = scale_merge_spec.scale_spectrum(template_spec, spec, spectrum_to_scale=2)
                    plt.plot(scaled_spec.wavelength, scaled_spec.flux, label=spec.filename)
            plt.plot(template_spec.wavelength, template_spec.flux, label = 'temp-{}'.format(template_spec.filename))
            plt.legend(loc='best', fontsize='x-small')
            plt.title(date)
            plt.xlim(xmax=10000)
            plt.savefig(os.path.join(FIG_DIR, 'wiserep_spectra_{}_{}.pdf'.format(sn,date)))
            plt.close()
    plt.ion()


# Combine Similar Spectra

complicated spectra combined with scombine in scombine_spectra notebook

In [45]:
def create_simple_comb_template_spec(sn_dir, sn, combine_spec_dict, redshift):
    for date in combine_spec_dict.keys():
        date_dict = combine_spec_dict[date]
        if isinstance(date_dict['filename'], str):
            spec = scale_merge_spec.read_spec(sn_dir, date_dict['filename'])
            spec = scale_merge_spec.trim_wavelengths(spec)
            wl = spec.wavelength
            flux = spec.flux
        elif date_dict['scombine'] is False:
            assert len(date_dict['filename'])==2
            spec1 = scale_merge_spec.read_spec(sn_dir, date_dict['filename'][0])
            spec2 = scale_merge_spec.read_spec(sn_dir, date_dict['filename'][1])
            spec1, spec2 = scale_merge_spec.scale_spectrum(spec1, spec2)
            wl, flux = scale_merge_spec.median_combine_data([spec1, spec2])
        else:
            continue
        #deredshift
        #wl = spectroscopy.apply_redshift(wl, redshift)
        tbdata = Table([wl, flux], names=['col0', 'col1'])
        tbdata.write('../SNID_templates/{}_{}_combine.dat'.format(sn, date),  
                     overwrite=True,
                     format='ascii.fixed_width_no_header',
                    delimiter=' ',
                     delimiter_pad=None,
                     bookend=False,
                    formats={'col0':'{0: >13.2F}', 'col1':'{: >13.6E}'})

# Test Combination

In [46]:
def plot_combined_spec(sn_dir, sn, combine_spec_dict):
    for date in combine_spec_dict.keys():
        if combine_spec_dict[date]['scombine'] is False:
            plt.figure()
            date_dict = combine_spec_dict[date]
            if isinstance(date_dict['filename'], str):
                spec = scale_merge_spec.read_spec(sn_dir, date_dict['filename'])
                plt.plot(spec.wavelength, spec.flux, label = 'untrimmed')
            else:
                fname1, fname2=date_dict['filename']
                spec1 = scale_merge_spec.read_spec(sn_dir, fname1)
                spec2 = scale_merge_spec.read_spec(sn_dir, fname2)
                spec1, spec2 = scale_merge_spec.scale_spectrum(spec1, spec2)
                plt.plot(spec1.wavelength, spec1.flux, label = spec1.filename)
                plt.plot(spec2.wavelength, spec2.flux, label = spec2.filename)
            final_spec = scale_merge_spec.read_spec('../SNID_templates','{}_{}_combine.dat'.format(sn,date))
            plt.plot(final_spec.wavelength, final_spec.flux, 'k', label='combined')
            plt.legend(loc='best', fontsize='x-small')
            plt.xlabel('wavelength')
            plt.ylabel('flux')
            plt.title('Combined Spectra for {}'.format(date))
            plt.savefig(os.path.join(FIG_DIR, '{}_{}_combined.pdf'.format(sn,date)))
            plt.close()
                

In [47]:
tbdata = ascii.read(os.path.join(CAT_DIR, '2013ej_cat.csv'))
sn_dir = '../2013ej'
sn='sn13ej'
dist = 9.7*u.Mpc
redshift = WMAP9.H0*dist/c.c.to(u.km/u.s)
print(redshift)

0.002242898318676182


In [ ]:
plot_scaled_spectra(sn_dir, sn, tbdata)
#Copy and paste output below and consult plots to help make combine_spec_dict

## Create Array for use:

In [48]:
combine_spec_dict = {'2013-07-27':{'filename':('2013ej_20130727_fts.fits'), 'scombine':False},
                    '2013-07-31':{'filename':('2013ej_20130731_P122_BC_300tr.fits'), 'scombine':False},
                    '2013-08-01':{'filename':('2013ej_20130801_ftn.fits'), 'scombine':False},
                    '2013-08-02':{'filename':('2013ej_20130802_ftn.fits',
                                              '2013ej_20130802_fts.fits'), 'scombine':False},
                    '2013-08-03':{'filename':('2013ej_20130803_fts.fits'), 'scombine':False},
                    '2013-08-04':{'filename':('sn2013ej-20130804.383-r.flm', 
                                              '2013ej_20130804_fts.fits',
                                              'sn2013ej-20130804.511-ui.flm'), 'scombine':True},
                    '2013-08-05':{'filename':('2013ej_20130805_ftn.fits',
                                              '2013ej_20130805_fts.fits',
                                              'SN2013ej_20130804_Gr11_Free_slit1.0_56999_1.fits',
                                              'SN2013ej_20130804_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':True},
                    '2013-08-07':{'filename':('2013ej_20130807_P122_BC_300tr.fits'), 'scombine':False},
                    '2013-08-08':{'filename':('2013ej_20130808_fts.fits',
                                              '2013ej_20130808_P122_BC_300tr.fits'), 'scombine':True},
                    '2013-08-09':{'filename':('2013ej_20130809_fts.fits'), 'scombine':False},
                    '2013-08-10':{'filename':('2013ej_20130810_fts.fits'), 'scombine':False},
                    '2013-08-11':{'filename':('2013ej_20130811_fts.fits', 
                                              'SN2013ej_20130811_wifes_B.fits',
                                              'SN2013ej_20130811_wifes_R.fits'), 'scombine':True},
                    '2013-08-12':{'filename':('2013ej_20130812_fts.fits',
                                              'sn2013ej-20130812.508-ui.flm'), 'scombine':True},
                    '2013-08-13':{'filename':('2013ej_20130813_fts.fits'), 'scombine':False},
                    '2013-08-15':{'filename':('2013ej_20130815_fts.fits',
                                              'SN2013ej_20130814_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':True},
                     #Spectra shape inconsistent with day before and day after
                    #'2013-08-16':{'filename':('SN2013ej_20130815_Gr11_Free_slit1.0_56999_1.fits',
                    #                          'SN2013ej_20130815_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-08-17':{'filename':('2013ej_20130817_fts.fits'), 'scombine':False},
                    '2013-08-19':{'filename':('SN2013ej_20130819_wifes_B.fits',
                                              'SN2013ej_20130819_wifes_R.fits'), 'scombine':False},
                    '2013-08-27':{'filename':('SN2013ej_20130826_Gr11_Free_slit1.0_56999_1.fits',
                                              'SN2013ej_20130826_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-08-30':{'filename':('SN2013ej_20130829_Gr11_Free_slit1.0_56999_1.fits',
                                              'SN2013ej_20130829_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-09-06':{'filename':('sn2013ej-20130906.414-ui.flm'), 'scombine':False},
                    '2013-09-09':{'filename':('SN2013ej_20130908_Gr11_Free_slit1.5_56999_1.fits',
                                              'SN2013ej_20130908_Gr16_OG530_slit1.5_56999_1.fits'), 'scombine':False},
                    '2013-09-10':{'filename':('sn2013ej-20130910.598-hal.flm'), 'scombine':False},
                    '2013-09-13':{'filename':('SN2013ej_20130912_Gr11_Free_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-09-19':{'filename':('SN2013ej_20130919_wifes_B.fits',
                                              'SN2013ej_20130919_wifes_R.fits'), 'scombine':False},
                    '2013-10-01':{'filename':('sn2013ej-20131001.548-ui.flm'), 'scombine':False},
                    '2013-10-03':{'filename':('SN2013ej_20131002_Gr11_Free_slit1.0_57001_1.fits',
                                              'SN2013ej_20131002_Gr16_OG530_slit1.0_57001_1.fits'), 'scombine':False},
                    '2013-10-05':{'filename':('sn2013ej-20131005.329-ui.flm'), 'scombine':False},
                    '2013-10-08':{'filename':('sn2013ej-20131008.483-hal.flm'), 'scombine':False},
                    '2013-10-10':{'filename':('sn2013ej-20131010.291-ui.flm'), 'scombine':False},
                    '2013-10-16':{'filename':('SN2013ej_20131016_wifes_B.fits',
                                              'SN2013ej_20131016_wifes_R.fits'), 'scombine':False},
                    '2013-10-24':{'filename':('SN2013ej_20131024_wifes_B.fits',
                                              'SN2013ej_20131024_wifes_R.fits'), 'scombine':False},
                    '2013-10-26':{'filename':('sn2013ej-20131026.264-ui.flm'), 'scombine':False},
                    '2013-10-27':{'filename':('SN2013ej_20131026_Gr11_Free_slit1.0_57001_1.fits',
                                              'SN2013ej_20131026_Gr16_OG530_slit1.0_57001_1.fits'), 'scombine':False},
                    '2013-11-02':{'filename':('sn2013ej-20131102.341-ui.flm'), 'scombine':False},
                    '2013-11-08':{'filename':('sn2013ej-20131108.312-ui.flm'), 'scombine':False},
                    '2013-11-24':{'filename':('SN2013ej_20131123_Gr11_Free_slit1.0_57002_1.fits',
                                              'SN2013ej_20131123_Gr16_OG530_slit1.0_57002_1.fits'), 'scombine':False},
                    '2013-11-28':{'filename':('sn2013ej-20131128.367-ui.flm'), 'scombine':False},
                    '2013-12-06':{'filename':('sn2013ej-20131206.394-ui.flm'), 'scombine':False},
                    '2013-12-24':{'filename':('SN2013ej_20131223_Gr11_Free_slit1.0_57002_1.fits',
                                              'SN2013ej_20131223_Gr16_OG530_slit1.0_57002_1.fits'), 'scombine':False},
                    '2014-01-24':{'filename':('SN2013ej_20140123_Gr11_Free_slit1.0_57003_1.fits'), 'scombine':False},
                    '2014-01-31':{'filename':('SN2013ej_20140130_Gr16_OG530_slit1.5_57003_1.fits'), 'scombine':False}}

#ofile = open('../2013ej/combine_dict.pkl', 'wb')
#pickle.dump(combine_spec_dict, ofile)
#ofile.close()

In [49]:
create_simple_comb_template_spec(sn_dir, sn, combine_spec_dict, redshift)

WARNING, Disperions differ by more than 10\% from average 1.0137129741003719
[ 0.77454547  1.25288047]
SN2013ej_20130919_wifes_B.fits
WARNING, Disperions differ by more than 10\% from average 1.014807084208769
[ 0.77706505  1.25254912]
SN2013ej_20131024_wifes_B.fits
WARNING, Disperions differ by more than 10\% from average 1.0138531255997805
[ 0.77457853  1.25312772]
SN2013ej_20130819_wifes_B.fits
WARNING, Disperions differ by more than 10\% from average 1.0114549926004202
[ 0.76876955  1.25414043]
SN2013ej_20131016_wifes_B.fits


In [50]:
plot_combined_spec(sn_dir, sn, combine_spec_dict)

# Now Run scombine_spectra notebook

# 2012A

In [51]:
#Almost no data in wiserep. files that do exist are not for 2012A, but 2012a?
tbdata = ascii.read(os.path.join(CAT_DIR, '2012A_cat.csv'))
sn_dir = '../2012A'
sn='sn12A'

dist = 9.8*u.Mpc #Tomasella, 2013
redshift = WMAP9.H0*dist/c.c.to(u.km/u.s)
print(redshift)

0.002266020981755318


In [41]:
plot_scaled_spectra(sn_dir, sn, tbdata)
#Copy and paste output below and consult plots to help make combine_spec_dict

'2013-03-15':{'filename':(
'sn2012a-20130315.145-r.flm',


In [52]:
combine_spec_dict = {
    '2012-01-10':{'filename':('2012A_20120110.fits'), 'scombine':False},
    '2012-01-13':{'filename':('2012A_20120113.fits'), 'scombine':False},
    '2012-01-15':{'filename':('2012A_20120115.fits'), 'scombine':False},
    '2012-01-17':{'filename':('2012A_20120117.fits'), 'scombine':False},
    '2012-01-18':{'filename':('2012A_20120118a.fits','2012A_20120118.fits'), 'scombine':False},
    '2012-01-20':{'filename':('2012A_20120120.fits'), 'scombine':False},
    '2012-01-21':{'filename':('2012A_20120121.fits'), 'scombine':False},
    '2012-01-22':{'filename':('2012A_20120122.fits'), 'scombine':False},
    '2012-01-23':{'filename':('2012A_20120123.fits'), 'scombine':False},
    '2012-01-24':{'filename':('2012A_20120124.fits'), 'scombine':False},
    '2012-01-26':{'filename':('2012A_20120126.fits'), 'scombine':False},
    '2012-01-27':{'filename':('2012A_20120127.fits'), 'scombine':False},
    '2012-01-29':{'filename':('2012A_20120129.fits'), 'scombine':False},
    '2012-01-30':{'filename':('2012A_20120130.fits'), 'scombine':False},
    '2012-02-05':{'filename':('2012A_20120205.fits'), 'scombine':False},
    '2012-02-11':{'filename':('2012A_20120211.fits'), 'scombine':False},
    '2012-02-13':{'filename':('2012A_20120213.fits'), 'scombine':False},
    '2012-02-18':{'filename':('2012A_20120218.fits'), 'scombine':False},
    '2012-02-23':{'filename':('2012A_20120223.fits'), 'scombine':False},
    '2012-02-25':{'filename':('2012A_20120225.fits'), 'scombine':False},
    '2012-02-28':{'filename':('2012A_20120228.fits'), 'scombine':False},
    '2012-03-12':{'filename':('2012A_20120312.fits'), 'scombine':False},
    '2012-03-13':{'filename':('2012A_20120313.fits'), 'scombine':False},
    #'2012-03-14':{'filename':('2012A_20120314.fits'), 'scombine':False}, #primarily IR
    '2012-03-15':{'filename':('2012A_20120315.fits'), 'scombine':False},
    '2012-03-17':{'filename':('2012A_20120317.fits'), 'scombine':False},
    '2012-03-26':{'filename':('2012A_20120326.fits'), 'scombine':False},
    '2012-03-27':{'filename':('2012A_20120327.fits'), 'scombine':False},
    '2012-03-29':{'filename':('2012A_20120329.fits'), 'scombine':False},
    '2012-03-31':{'filename':('2012A_20120331.fits'), 'scombine':False},
    '2012-04-12':{'filename':('2012A_20120412.fits'), 'scombine':False},
    '2012-04-23':{'filename':('2012A_20120423.fits'), 'scombine':False},
    '2012-04-24':{'filename':('2012A_20120424.fits'), 'scombine':False},
    '2012-04-27':{'filename':('2012A_20120427.fits'), 'scombine':False},
    '2012-05-01':{'filename':('2012A_20120501.fits'), 'scombine':False},
    '2012-05-15':{'filename':('2012A_20120515a.fits','2012A_20120515.fits'), 'scombine':False},
    '2012-05-29':{'filename':('2012A_20120529.fits'), 'scombine':False},
    '2012-06-08':{'filename':('2012A_20120608.fits'), 'scombine':False},
    #'2012-06-27':{'filename':('2012A_20120627.fits'), 'scombine':False},  #Noisy spectrum
    '2013-02-03':{'filename':('2012A_20130203.fits'), 'scombine':False},
    '2013-03-15':{'filename':('sn2012a-20130315.145-r.flm'), 'scombine':False}
}

In [53]:
create_simple_comb_template_spec(sn_dir, sn, combine_spec_dict, redshift)

WARNING, Disperions differ by more than 10\% from average 2.884848793356923
[ 1.          4.76969759]
2012A_20120118a.fits


In [54]:
plot_combined_spec(sn_dir, sn, combine_spec_dict)

# 2013by

In [18]:
#Almost no data in wiserep. files that do exist are not for 2012A, but 2012a?
tbdata = ascii.read(os.path.join(CAT_DIR, '2013by_cat.csv'))
sn_dir = '../2013by'
sn='sn13by'
plot_scaled_spectra(sn_dir, sn, tbdata)
#Copy and paste output below and consult plots to help make combine_spec_dict

'2013-04-24':{'filename':(
'2013-05-08':{'filename':(
'2013-05-22':{'filename':(
'2013-05-26':{'filename':(
'SN2013by_20130526_wifes_B.fits',
'SN2013by_20130526_wifes_R.fits',
'2013-07-29':{'filename':(
'2013-08-19':{'filename':(
'SN2013by_20130819_wifes_B.fits',
'SN2013by_20130819_wifes_R.fits',
'2014-07-08':{'filename':(
'SN2013by_20140708_wifes_B.fits',
'SN2013by_20140708_wifes_R.fits',


Only 3 epochs, one with good spectra